In [1]:
import time
import torch


def test_process(queue):
    print(f"Test process started")
    try:
        while True:
            time.sleep(.4)
            dummy_t = torch.ones(30,30,30)
            queue.put(dummy_t)
            print(f"===== TEST PROCESS PUT ::: {dummy_t.shape}")
    except Exception as e:
        print(f"Exception in test process: {e}")


if __name__ == '__main__':
    # Create a child SimpleQueue and a child process
    torch.set_default_device("cuda")
    queue = torch.multiprocessing.SimpleQueue()
    test_process = torch.multiprocessing.Process(target=test_process, args=(queue,))
    test_process.start()

    while True:
        if not queue.empty():
            print(f"-------->>> MAIN PROCESS GOT::: {queue.get()}")
        else:
            time.sleep(.001)


KeyboardInterrupt: 

In [ ]:
# DOES NOT WORK! 
import time
import torch

from mlagents import simple_queue_with_size

def test_producer_func(queue):
    torch.set_default_device("cuda")
    print(f"Test process started")
    try:
        while True:
            dummy_t = torch.ones(1000, 30,30,30)
            queue.put(dummy_t)
            print(f"===== TEST PROCESS PUT ::: {dummy_t.shape}")
            time.sleep(5)
    except Exception as e:
        print(f"Exception in test process: {e}")


def test_trainer_func(trainer):
    torch.set_default_device("cuda")
    queue = simple_queue_with_size.SimpleQueueWithSize()
    test_process = torch.multiprocessing.Process(target=test_producer_func, args=(queue,))
    test_process.start()
    if not queue.empty():
        print(f"-------->>> MAIN PROCESS GOT::: {queue.get()}")
    else:
        time.sleep(.001)

    # trainer._initialize()
    # while True:
    #     trainer.advance_consumer()


class Trainer:
    def _initialize(self):
        self.queue = simple_queue_with_size.SimpleQueueWithSize()
        test_process = torch.multiprocessing.Process(target=test_producer_func, args=(self.queue,))
        test_process.start()

    def advance_consumer(self):
        if not self.queue.empty():
            print(f"-------->>> MAIN PROCESS GOT::: {self.queue.get()}")
        else:
            time.sleep(.001)



if __name__ == '__main__':
    # Create a child SimpleQueue and a child process
    torch.set_default_device("cuda")
    trainer = Trainer()
    trainer_process = torch.multiprocessing.Process(target=test_trainer_func, args=(trainer,))
    trainer_process.start()



In [ ]:
# ALSO DOES NOT WORK!

# DOES NOT WORK! 
import time
import torch

from mlagents import simple_queue_with_size

def test_producer_func(queue):
    torch.set_default_device("cuda")
    print(f"Test process started")
    try:
        while True:
            dummy_t = torch.ones(1000, 30,30,30)
            queue.put(dummy_t)
            print(f"===== TEST PROCESS PUT ::: {dummy_t.shape}")
            time.sleep(5)
    except Exception as e:
        print(f"Exception in test process: {e}")


def test_trainer_func(trainer):
    torch.set_default_device("cuda")
    queue = simple_queue_with_size.SimpleQueueWithSize()
    test_process = torch.multiprocessing.Process(target=test_producer_func, args=(queue,))
    test_process.start()
    if not queue.empty():
        print(f"-------->>> MAIN PROCESS GOT::: {queue.get()}")
    else:
        time.sleep(.001)

    # trainer._initialize()
    # while True:
    #     trainer.advance_consumer()


class Trainer:
    def _initialize(self):
        self.queue = simple_queue_with_size.SimpleQueueWithSize()
        test_process = torch.multiprocessing.Process(target=test_producer_func, args=(self.queue,))
        test_process.start()

    def advance_consumer(self):
        if not self.queue.empty():
            print(f"-------->>> MAIN PROCESS GOT::: {self.queue.get()}")
        else:
            time.sleep(.001)



if __name__ == '__main__':
    # Create a child SimpleQueue and a child process
    torch.set_default_device("cuda")
    trainer = Trainer()
    trainer_process = torch.multiprocessing.Process(target=test_trainer_func, args=(trainer,))
    trainer_process.start()

In [5]:
# WEIRD ASS BEHAVIOR, STARTS WORKING AFTER A FEW ITERATIIONS: 
from tabnanny import check
import time
import torch

from mlagents import simple_queue_with_size
from mlagents.trainers.torch_entities.utils import ModelUtils


def test_producer_func(queue):
    torch.set_default_device("cuda")
    print(f"Test process started")
    try:
        while True:
            time.sleep(5)
            dummy_t = torch.ones(1000, 30,30,30)
            non_ones = torch.sum(dummy_t == 1)
            queue.put(dummy_t)
            print(f"===== TEST PROCESS PUT ::: {dummy_t.shape} with num non ones: {torch.sum(dummy_t == 1)} non ones before: {non_ones} ")
    except Exception as e:
        print(f"Exception in test process: {e}")


def test_trainer_func(trainer):
    torch.set_default_device("cuda")
    trainer._initialize()
    while True:
        trainer.advance_consumer()


class Trainer:
    def _initialize(self):
        self.queue = simple_queue_with_size.SimpleQueueWithSize()
        test_process = torch.multiprocessing.Process(target=test_producer_func, args=(self.queue,))
        test_process.start()

    def advance_consumer(self):
        if not self.queue.empty():
            item = self.queue.get()
            print(f"-------->>> MAIN PROCESS GOT TENSOR num non ones::: {torch.sum(item == 1)}")
            # print(f"-------->>> MAIN PROCESS GOT::: {self.queue.get()}")
        else:
            time.sleep(.001)



if __name__ == '__main__':
    # Create a child SimpleQueue and a child process
    torch.set_default_device("cuda")
    trainer = Trainer()
    trainer_process = torch.multiprocessing.Process(target=test_trainer_func, args=(trainer,))
    trainer_process.start()

# OUTPUTS 
"""
(mlagents-20-v2) E:\Unity Projects\python_env\mlagents-20-clone\ml-agents>python send_cuda_test.py
Test process started
===== TEST PROCESS PUT ::: torch.Size([1000, 30, 30, 30]) with num non ones: 0 non ones before: 27000000
-------->>> MAIN PROCESS GOT TENSOR num non ones::: 0
===== TEST PROCESS PUT ::: torch.Size([1000, 30, 30, 30]) with num non ones: 0 non ones before: 27000000
-------->>> MAIN PROCESS GOT TENSOR num non ones::: 0
-------->>> MAIN PROCESS GOT TENSOR num non ones::: 27000000
===== TEST PROCESS PUT ::: torch.Size([1000, 30, 30, 30]) with num non ones: 27000000 non ones before: 27000000
-------->>> MAIN PROCESS GOT TENSOR num non ones::: 27000000
===== TEST PROCESS PUT ::: torch.Size([1000, 30, 30, 30]) with num non ones: 27000000 non ones before: 27000000
"""

SyntaxError: EOF while scanning triple-quoted string literal (3734864581.py, line 65)

In [6]:
# DOESN'T WORK WITH SMALLER TENSORS EITHER - 
from tabnanny import check
import time
import torch

from mlagents.simple_queue_with_size import SimpleQueueWithSize
from mlagents.trainers.torch_entities.utils import ModelUtils


def test_producer_func(queue : torch.multiprocessing.SimpleQueue):
    torch.set_default_device("cuda")
    print(f"Test process started")
    try:
        while True:
            time.sleep(5)
            dummy_t = torch.ones(1, 1,1,30)
            non_ones = torch.sum(dummy_t == 1)
            queue.put(dummy_t)
            print(f"===== TEST PROCESS PUT ::: {dummy_t.shape} with num non ones: {torch.sum(dummy_t == 1)} non ones before: {non_ones} ")
    except Exception as e:
        print(f"Exception in test process: {e}")


def test_trainer_func(trainer):
    torch.set_default_device("cuda")
    queue = torch.multiprocessing.SimpleQueue()
    test_process = torch.multiprocessing.Process(target=test_producer_func, args=(queue,))
    test_process.start()
    # trainer._initialize()
    while True:
        item = queue.get()
        print(f"-------->>> MAIN PROCESS GOT TENSOR num non ones::: {torch.sum(item == 1)}")
        # print(f"-------->>> MAIN PROCESS GOT::: {self.queue.get()}")
        time.sleep(.001)

    #     trainer.advance_consumer()


class Trainer:
    def _initialize(self):
        self.queue = torch.multiprocessing.SimpleQueue()
        test_process = torch.multiprocessing.Process(target=test_producer_func, args=(self.queue,))#
        test_process.start()

    def advance_consumer(self):
        if not self.queue.empty():
            item = self.queue.get()
            print(f"-------->>> MAIN PROCESS GOT TENSOR num non ones::: {torch.sum(item == 1)}")
            # print(f"-------->>> MAIN PROCESS GOT::: {self.queue.get()}")
        else:
            time.sleep(.001)



if __name__ == '__main__':
    # Create a child SimpleQueue and a child process
    torch.set_default_device("cuda")
    trainer = Trainer()
    trainer_process = torch.multiprocessing.Process(target=test_trainer_func, args=(trainer,))
    trainer_process.start()

# OUTPUTS
"""
(mlagents-20-v2) E:\Unity Projects\python_env\mlagents-20-clone\ml-agents>python send_cuda_test.py
Test process started
===== TEST PROCESS PUT ::: torch.Size([1, 1, 1, 30]) with num non ones: 0 non ones before: 0
-------->>> MAIN PROCESS GOT TENSOR num non ones::: 0
===== TEST PROCESS PUT ::: torch.Size([1, 1, 1, 30]) with num non ones: 30 non ones before: 30
-------->>> MAIN PROCESS GOT TENSOR num non ones::: 30
===== TEST PROCESS PUT ::: torch.Size([1, 1, 1, 30]) with num non ones: 30 non ones before: 30
-------->>> MAIN PROCESS GOT TENSOR num non ones::: 30
"""


In [ ]:
# PYTROCH MINIMAL REPRO
import time
import torch

def test_producer_func(queue : torch.multiprocessing.SimpleQueue):
    torch.set_default_device("cuda")
    while True:
        dummy_t = torch.ones(1, 1,1,30)
        non_ones = torch.sum(torch.isclose(dummy_t, torch.ones(1)))# torch.sum(dummy_t == 1)
        queue.put(dummy_t)
        print(f"===== TEST PROCESS PUT ::: {dummy_t.shape} with num non ones: {torch.sum(torch.isclose(dummy_t, torch.ones(1)))} non ones before: {non_ones} ")


if __name__ == '__main__':
    torch.set_default_device("cuda")
    queue = torch.multiprocessing.SimpleQueue()
    producer_process = torch.multiprocessing.Process(target=test_producer_func, args=(queue,))
    producer_process.start()
    while True:
        if not queue.empty():
            item = queue.get()
            print(f"-------->>> MAIN PROCESS GOT TENSOR num non ones::: {torch.sum(torch.isclose(item, torch.ones(1)))}")
        else:
            time.sleep(.001)




In [ ]:
# VICE VERSA DOES NOT WORK EITHER 


def test_producer_func(queue : torch.multiprocessing.SimpleQueue):
    torch.set_default_device("cuda")
    while True:
        if not queue.empty():
            item = queue.get()
            print(f"-------->>> test_producer_func TENSOR num non ones::: {torch.sum(item == 1)}")
        else:
            time.sleep(.001)
        # dummy_t = torch.ones(1, 1,1,30)
        # non_ones = torch.sum(dummy_t == 1)
        # queue.put(dummy_t)
        # print(f"===== TEST PROCESS PUT ::: {dummy_t.shape} with num non ones: {torch.sum(dummy_t == 1)} non ones before: {non_ones} ")
        # time.sleep(3)


if __name__ == '__main__':
    torch.set_default_device("cuda")
    queue = torch.multiprocessing.SimpleQueue()
    producer_process = torch.multiprocessing.Process(target=test_producer_func, args=(queue,))
    producer_process.start()
    while True:
        # if not queue.empty():
        #     item = queue.get()
        #     print(f"-------->>> MAIN PROCESS GOT TENSOR num non ones::: {torch.sum(item == 1)}")
        # else:
        #     time.sleep(.001)
        dummy_t = torch.ones(1, 1,1,30)
        non_ones = torch.sum(dummy_t == 1)
        queue.put(dummy_t)
        print(f"===== __main__ PROCESS PUT ::: {dummy_t.shape} with num non ones: {torch.sum(dummy_t == 1)} non ones before: {non_ones} ")
        time.sleep(3)